In [9]:
import bonobo
from requests_html import HTMLSession
from pprint import pprint
import dotenv
import os
import json
import pandas as pd
from geopy import Nominatim

import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)

dotenv.load_dotenv()

True

In [2]:
geocoder = Nominatim(user_agent='Kiga Graz')

In [71]:
center = geocoder.geocode('Hauptplatz 1, 8010 Graz')
home = geocoder.geocode('Grabenstrasse 45, Graz')
hbf = geocoder.geocode('hauptbahnhof, Graz')
uni = geocoder.geocode('Universitätsplatz 3, 8010 Graz')
tu_uni = geocoder.geocode('Rechbauerstraße 12, 8010 Graz')
kc = geocoder.geocode('Inffeldgasse 13, 8010 Graz')
lkh = geocoder.geocode('Auenbruggerplatz 1, 8036 Graz')

In [73]:
lkh

Location(1, Auenbruggerplatz, Geidorf, Graz, Steiermark, 8036, Österreich, (47.0794539, 15.4657707, 0.0))

In [10]:
mapbox_access_token = os.getenv('MAPBOX')
mapbox_access_token

'pk.eyJ1Ijoic2xhZGtvdm0iLCJhIjoiY2piZjh3eWluMnMyZjMzbXJydWF2MWtzdSJ9.Ltqoa_kxJ1EMVj-8qGFNCA'

In [11]:
with open('krippe_stad.json') as f:
    krippe_stad = json.load(f)
with open('krippe_privat.json') as f:
    krippe_privat = json.load(f)

In [12]:
df_stad = pd.DataFrame(krippe_stad)
df_privat = pd.DataFrame(krippe_privat)

In [74]:
data = [
    go.Scattermapbox(
        name='Stadt',
        lat=df_stad.lat,
        lon=df_stad.lon,
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=df_stad.GT*5,
            color='red'
        ),
        text=df_stad.name,
    ), 
    
    go.Scattermapbox(
        name='Privat',
        lat=df_privat.lat,
        lon=df_privat.lon,
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=10,
            color='blue'
        ),
        text=df_privat.name,
    ),
    
    go.Scattermapbox(
        name='Landmarks',
        lat=[center.latitude, home.latitude, hbf.latitude, uni.latitude, tu_uni.latitude, kc.latitude, lkh.latitude],
        lon=[center.longitude, home.longitude, hbf.longitude, uni.longitude, tu_uni.longitude, kc.longitude, lkh.longitude],
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=10,
            color='green'
        ),
        text='landmarks',
    )
]



layout = go.Layout(
    title='Kinderkrippe Graz 2019',
    autosize=True,
    width=1024,
    height=1024,
    legend=dict(orientation="h", x=0, y=0),
    hovermode='closest',
    mapbox=go.layout.Mapbox(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=go.layout.mapbox.Center(
            lat=center.latitude,
            lon=center.longitude
        ),
        pitch=0,
        zoom=12
    )
)

fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='Kiderkrippe Graz')

In [38]:
layout

Layout({
    'autosize': True,
    'height': 1024,
    'hovermode': 'closest',
    'legend': {'orientation': 'h', 'x': 0, 'y': 0},
    'mapbox': {'accesstoken': ('pk.eyJ1Ijoic2xhZGtvdm0iLCJhIjo' ... 'WtzdSJ9.Ltqoa_kxJ1EMVj-8qGFNCA'),
               'bearing': 0,
               'center': {'lat': 47.0704298, 'lon': 15.4382352},
               'pitch': 0,
               'zoom': 12},
    'shapes': [{'line': {'color': 'rgb(128, 0, 128)', 'dash': 'dot', 'width': 4},
                'type': 'line',
                'x0': 47.0704298,
                'x1': 47.0844163,
                'xref': 'x',
                'y0': 15.4382352,
                'y1': 15.4366836263287,
                'yref': 'y'}],
    'title': {'text': 'Kinderkrippe Graz 2019'},
    'width': 1024
})

In [24]:
print(home.latitude, center.latitude)
print(home.longitude, center.longitude)

47.0844163 47.0710049
15.4366836263287 15.4385152547851


In [43]:
fig['layout']['mapbox']

layout.Mapbox({
    'accesstoken': ('pk.eyJ1Ijoic2xhZGtvdm0iLCJhIjo' ... 'WtzdSJ9.Ltqoa_kxJ1EMVj-8qGFNCA'),
    'bearing': 0,
    'center': {'lat': 47.0704298, 'lon': 15.4382352},
    'pitch': 0,
    'zoom': 12
})